In [1]:
%matplotlib inline

import os
import sys
import re
import random
import csv
import concurrent.futures
import collections
from collections import deque
import ujson as json
import itertools


from skyway.canvas import WNUTM5kmTiling, ProjectedUTMTiling, GeoTile, WGS84Transformer, WORLD_CRS, MAP_CRS
from skyway.utils import itrreduce, to_gjson
from skyway.query import QueryBuilder, opf, NodeWayRelationQuery
from skyway.query.nominatim import Nominatim
from skyway.query.scrape import scrape_primary_features
import tiletanic as tt

import pandas as pd
import geopandas as gpd
import rasterio
import rioxarray as riox
import numpy as np
import shapely.ops as ops
import shapely.geometry as geom
from shapely.strtree import STRtree
import skimage.io as io

import vaex as vx

import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive
from IPython.display import display
import qgrid
from ipyleaflet import (
    Map,
    basemaps,
    basemap_to_tiles,
    Layer, LayerGroup,
    Marker, MarkerCluster,
    Polyline, Polygon, GeoJSON,
    WidgetControl, ScaleControl, LayersControl, SearchControl
)
from sidecar import Sidecar

import time
from tqdm.notebook import tqdm

import visdom

In [ ]:
vis = visdom.Visdom(port=8087)

In [2]:
DATA_PATH = "/home/ubuntu/data/ard"
COG_PATH = "/home/ubuntu/data/ard/33"
CHIP_PATH = "/home/ubuntu/data/chips/33"

qkp = re.compile('^\d{12}$')

In [3]:
def iter_quadkey_paths(zone, data_path=DATA_PATH):
    for item in os.listdir(os.path.join(data_path, str(zone))):
        if qkp.match(item):
            yield qk         
        
        
def setup_query():
    nwr_q = NodeWayRelationQuery()
    qb = QueryBuilder()
    qb.include_geometries()
    qb.settings.payload_format = 'json'
    qb.settings.maxsize = int(qb.settings.MAXSIZE_LIMIT / 2)
    qb.qsx.append(nwr_q)
    return qb

    
def setup_tiler(utm_zone):
    tiler = WNUTM5kmTiling()
    proj = ProjectedUTMTiling(zone=utm_zone, tiler=tiler)
    return proj
 
    
def fetch_osm_by_quadkey(zone, data_path):
    data_region_path = os.path.join(data_path, str(zone))
    qkdeq = collections.deque(os.listdir(data_region_path))
    
    node_tags = collections.defaultdict(list)
    way_tags = collections.defaultdict(list)
    rel_tags = collections.defaultdict(list)
    
    qb = setup_query()
    proj = setup_tiler(zone)
    
    pbar = tqdm(total=len(qkdeq))
    while qkdeq:
        qk = qkdeq.pop()
        tile = proj.tile_from_quadkey(qk)
        tile.toWGS84()
        west, south, east, north = tile.bounds
        qb.GlobalBoundingBox = [south, west, north, east]
        print(f"Requesting: {qk}, {tile.bounds}")
        print("\n")

        try:
            r = qb.request()
            r.raise_for_status()
            res = r.json()
        except Exception as e:
            print(f"Got exception {e}, backing off and sleeping for one minute")
            qkdeq.append(qk)
            time.sleep(60)
            continue
        
        nodes = [elm for elm in res['elements'] if elm['type']=='node']
        nodes_w_tags = [elm for elm in nodes if elm.get("tags") is not None]
        ways = [elm for elm in res['elements'] if elm['type']=='way']
        ways_w_tags = [elm for elm in ways if elm.get("tags") is not None]
        rels = [elm for elm in res['elements'] if elm['type']=='relation']
        rels_w_tags = [elm for elm in rels if elm.get("tags") is not None]
        
        n_nodes = len(nodes)
        n_nodes_tagged = len(nodes_w_tags)
        n_ways = len(ways)
        n_ways_tagged = len(ways_w_tags)
        n_rels = len(rels)
        n_rels_tagged = len(rels_w_tags)
        
        print(f"Total elements returned: {len(res['elements'])}")
        
        print(f"Num tagged/total: Nodes({n_nodes_tagged}/{n_nodes}), Ways({n_ways_tagged}/{n_ways}), Rels({n_rels_tagged}/{n_rels})")
        print(f"Total items filtered: {n_nodes_tagged + n_ways_tagged + n_rels_tagged}")
        
        d = {"quadkey": qk, "nodes": nodes_w_tags, "ways": ways_w_tags, "relations": rels_w_tags}
        fp = os.path.join(os.path.join(data_region_path, qk), "osm_data.json")
        with open(fp, "w") as f:
            json.dump(d, f)
        print(f"wrote OSM data to {fp}")
        
        for elm in nodes_w_tags:
            for key, val in elm['tags'].items():
                node_tags[key].append(val)
        for elm in ways_w_tags:
            for key, val in elm['tags'].items():
                way_tags[key].append(val)
        for elm in rels_w_tags:
            for key, val in elm['tags'].items():
                rel_tags[key].append(val)
                
        print("\n")
        print("\n")
        print("\n")
        pbar.update(1)
        time.sleep(5)
        
    return (node_tags, way_tags, rel_tags) 
     
    
def build_tag_map(elm_type, zone, data_path):
    tag_map = collections.defaultdict(list)
    data_region_path = os.path.join(data_path, str(zone))
    quadkeys = [qk for qk in os.listdir(data_region_path) if "json" not in qk]
    
    for qk in quadkeys:
        fp = os.path.join(os.path.join(data_region_path, qk), "osm_data.json")
        with open(fp, "r") as f:
            data = json.load(f)
        elms = data[elm_type + "s"]
        for elm in elms:
            for key, val in elm['tags'].items():
                tag_map[key].append(val)
    write_path = os.path.join(data_region_path, f"{elm_type}_tags.json")
    with open(write_path, "w") as f:
        json.dump(tag_map, f)
    return tag_map


def get_children_at_zoom(parents, child_zoom):
    nodes = list()
    for parent in parents:
        if parent.zoom == child_zoom:
            return parents
        if parent.zoom > child_zoom:
            raise ValueError
        children = parent.children()
        nodes.extend(children)
    return get_children_at_zoom(nodes, child_zoom)


def get_cog_paths(zone, ard_path=DATA_PATH):
    ard_region_path = os.path.join(ard_path, str(zone))
    proj = setup_tiler(zone)
    quadkeys = [qk for qk in os.listdir(ard_region_path) if "json" not in qk]
    cog_paths = list()
    for qk in quadkeys:
        parent_tile = proj.tile_from_quadkey(qk)
        qk_ard_path = os.path.join(ard_region_path, qk)
        acq_dates = [dt for dt in os.listdir(qk_ard_path) if "json" not in dt]
        for acq_date in acq_dates:
            ard_path = os.path.join(qk_ard_path, acq_date)
            ard_items = os.listdir(ard_path)
            cogfiles = [item for item in ard_items if item[-11:] == "-visual.tif"]
            for cogfile in cogfiles:
                cog_paths.append(os.path.join(ard_path, cogfile))
    return cog_paths


def ard_image_path_from_chip(chip_fn):
    if chip_fn.endswith(".jpg"):
        chip_fn = chip_fn[:-4]
    qkhead, cat_id = chip_fn.split("_")
    qk_major = qkhead[4:16]
    cog_fn = f'''{cat_id}-visual.tif'''
    qk_cog_path = os.path.join(COG_PATH, qk_major)
    
    for date_dir in os.listdir(qk_cog_path):
        full_cog_path = os.path.join(os.path.join(qk_cog_path, date_dir), cog_fn)
        if os.path.exists(full_cog_path):
            return full_cog_path
    raise OSError("File not found")
        

In [4]:
def early_osm_inference():
    with open("/home/ubuntu/data/ard/33/node_tags.json") as f:
        node_tags = json.load(f)

    with open("/home/ubuntu/data/ard/33/way_tags.json") as f:
        way_tags = json.load(f)

    with open("/home/ubuntu/data/ard/33/rel_tags.json") as f:
        rel_tags = json.load(f)

    ntag_counts = {key: collections.Counter(val) for key, val in node_tags.items()}
    wtag_counts = {key: collections.Counter(val) for key, val in way_tags.items()}
    rtag_counts = {key: collections.Counter(val) for key, val in rel_tags.items()}

    node_keycounts = sorted([(key, len(val)) for key, val in node_tags.items()], key=lambda t: t[-1], reverse=True)
    way_keycounts = sorted([(key, len(val)) for key, val in way_tags.items()], key=lambda t: t[-1], reverse=True)
    rel_keycounts = sorted([(key, len(val)) for key, val in rel_tags.items()], key=lambda t: t[-1], reverse=True)

    #wtags_ordered = sorted([(key, wtag_counts[key].most_common(1)) for key, val in wtag_counts.items()], key=lambda t: t[-1][-1][-1], reverse=True)
    #wtags_ordered[:50]
    
    
## Load nodata scores map
def load_nodata_scores():
    with open("/home/ubuntu/data/chips/33/nodata_index.json", "r") as f:
        ndix = json.load(f)
    return ndix

In [5]:
#tags, tables = scrape_primary_features(up_to=True)

def osm_tables_to_kv(tables):
    legal = collections.defaultdict(lambda: collections.defaultdict(list))
    restricted = collections.defaultdict(lambda: collections.defaultdict(list))
    for topic, kvd in tables.items():
        for key, val_list in kvd.items():
            for val in val_list:
                try:
                    tag_val, desc = val
                    if " " in tag_val and tag_val.lower() != "user defined":
                        restricted[topic][key].append(tag_val)
                    else:
                        legal[topic][key].append(tag_val)
                except ValueError:
                    restricted[topic][key].append(val)
    return legal, restricted


def discriminate_tagschemes():            
    kvd_legal, kvd_restricted = osm_tables_to_kv(tables)
    kvd_legal.update(tags)
    

def get_all_tagkeys(keytag_dict):
    tagkeys = list()
    for key, val_list in keytag_dict.items():
        for val in val_list:
            tagkeys.append(val)
    return tagkeys


#tagkeys_legal = get_all_tagkeys(kvd_legal)
def compile_all_tagschema():
    all_tagkeys = [t for t in tagkeys_legal]
    for topic, keydict in kvd_restricted.items():
        all_tagkeys.extend(list(keydict.keys()))
    
    kvd_all = collections.defaultdict(lambda: collections.defaultdict(list))
    for d in [kvd_legal, kvd_restricted]:
        for topic, keydict in d.items():
            for tagkey, tagvals in keydict.items():
                for tagval in tagvals:
                    kvd_all[topic][tagkey].append(tagval)
    return kvd_all

def comile_all_primary_keys(kvd_all):
    all_primary_keys = list()
    for topic, keydict in kvd_all.items():
        for key in keydict:
            all_primary_keys.append(key)
    return all_primary_keys
        

def primary_key_element_stats(all_primary_keys):
    ntag_counts_f = {k: v for k, v in ntag_counts.items() if k in all_primary_keys}
    wtag_counts_f = {k: v for k, v in wtag_counts.items() if k in all_primary_keys}
    rtag_counts_f = {k: v for k, v in rtag_counts.items() if k in all_primary_keys}
    return (ntag_counts_f, wtag_counts_f, rtag_counts_f)
    
    
def tabulate_element_primary_stats(outfile="/home/ubuntu/data/osm/primary_keytags.csv", overwrite=False):
    if os.path.exists(outfile) and not overwrite:
        return False
    with open(outfile, "w") as f:
        writer = csv.writer(f)
        writer.writerow(["Topic", "TagKey", "NodeCount", "WayCount", "RelCount"])
        for topic, keydict in kvd_all.items():
            for key in keydict.keys():
                nc, wc, rc = (0, 0, 0)
                if ntag_counts_f.get(key):
                    nc = sum([t[-1] for t in ntag_counts_f[key].items()])
                if wtag_counts_f.get(key):
                    wc = sum([t[-1] for t in wtag_counts_f[key].items()])
                if rtag_counts_f.get(key):
                    rc = sum([t[-1] for t in rtag_counts_f[key].items()])
                writer.writerow([topic, key, nc, wc, rc])
                

def load_dataset_with_filtercol(infile="/home/ubuntu/data/osm/primary_keytags.csv"):
    key_dist = pd.read_csv(infile)
    default_selections = [True]*len(key_dist)
    key_dist['Selected'] = default_selections
    return key_dist


def filter_tagkeys_from_selection(key_dist_f):
    key_dist_f[key_dist_f['Selected'] == True]
    all_filtered_keys = list(key_dist_filtered.TagKey.values)
    return all_filtered_keys


def load_complete_primary_features_description(infile="/home/ubuntu/data/osm/primary_features.json"):
    with open(infile) as f:
        return json.load(f)
    

def filter_osm_by_region(osm_keys, out_file="primary_osm_data_filtered.json", in_file="osm_data.json", data_path=COG_PATH):
    quadkeys = [item for item in os.listdir(data_path) if qkp.match(item)]
    for qk in tqdm(quadkeys):
        print(qk)
        infile = os.path.join(os.path.join(data_path, qk), in_file)
        outfile = os.path.join(os.path.join(data_path, qk), out_file)
        filter_osm_data(osm_keys, qk, infile, outfile)
        print("\n")
    
def filter_osm_data(osm_keys, quadkey, infile, outfile):
    d = dict()
    d['quadkey'] = quadkey
    d['nodes'] = list()
    d['ways'] = list()
    d['relations'] = list()

    all_keys_set = set(osm_keys)

    with open(infile) as f:
        raw = json.load(f)
        for elm_type in ['nodes', "ways", "relations"]:
            raw_elms = raw[elm_type]
            print(f'''raw {elm_type} length: {len(raw_elms)}''')
            for elm in raw_elms:
                tags = elm.get('tags')
                elm_keys_set = set(list(tags.keys()))
                primary_intr = all_keys_set.intersection(elm_keys_set)
                if len(primary_intr) > 0:
                    new_elm = dict()
                    new_elm['type'] = elm['type']
                    new_elm['id'] = elm['id']
                    new_elm['tags'] = [{tk: elm['tags'][tk] for tk in primary_intr}]
                    if elm['type'] == "node":
                        new_elm['lat'] = elm['lat']
                        new_elm['lon'] = elm['lon']
                    if elm['type'] == 'way':
                        new_elm['geometry'] = elm['geometry']
                    if elm['type'] == 'relation':
                        new_elm['members'] = elm['members']
                    d[elm_type].append(elm)
        
                
    with open(outfile, "w") as ff:
        json.dump(d, ff)
        
    print(f'''filtered nodes length: {len(d['nodes'])}''')
    print(f'''filtered ways length: {len(d['ways'])}''')
    print(f'''filtered relations length: {len(d['relations'])}''')
    
# filter_osm_by_region(all_filtered_keys)

In [6]:
def get_primary_keys_filtered(infile="/home/ubuntu/data/osm/primary_keys_filtered.csv"):
    kvd_filt = pd.read_csv()
    kvd_filt = kvd_filt.drop(columns=["Unnamed: 0", "Selected"])
    return kvd_filt

In [7]:
def quadkey_regex(zoom=12):
    return re.compile(f"""'^\d{zoom}$'""")


def setup_tiler(utm_zone):
    tiler = WNUTM5kmTiling()
    proj = ProjectedUTMTiling(zone=utm_zone, tiler=tiler)
    return proj


def get_children_at_zoom(parents, child_zoom):
    nodes = list()
    for parent in parents:
        if parent.zoom == child_zoom:
            return parents
        if parent.zoom > child_zoom:
            raise ValueError
        children = parent.children()
        nodes.extend(children)
    return get_children_at_zoom(nodes, child_zoom)


def get_osm_elements(in_file, elm_type=None):
    with open(in_file) as f:
        d = json.load(f)
        if not elm_type:
            return d
        return d[elm_type]
    
    
def build_elm_geom(elmd):
    if elmd.get('type') == "node":
        return geom.Point((elmd['lon'], elmd['lat']))
    coords = elmd['geometry']
    geomtype = geom.LineString
    if coords[0] == coords[-1]:
        geomtype = geom.Polygon
    return geomtype([(p['lon'], p['lat']) for p in coords])

    
def build_osm_rtree(osm_file, elm_types=("nodes", "ways"), **kwargs):
    elements = get_osm_elements(osm_file, **kwargs)
    elm_geom_map = dict((elmd['id'], [build_elm_geom(elmd), elmd]) for 
                        elmtype in elm_types for elmd in elements[elmtype])
    # Include the osm data map so don't have to look up again downstream
    hash_lut = dict((id(elm_geom), osm_id) for osm_id, (elm_geom, elmd) in elm_geom_map.items())
    
    tree = STRtree([v[0] for v in elm_geom_map.values()])
    # Return the osm-id: (geom, osm payload element), the geom sys id: osm-id proxy and the rtree 
    return (elm_geom_map, hash_lut, tree)


def map_elms_to_tilechildren(utm_zone,
                             quadkey,
                             dst_zoom,
                             osm_filename="primary_osm_data_filtered.json",
                             primary_keys_path="/home/ubuntu/data/osm/primary_keys_filtered.csv"
                            ):
    
    osm_schema = get_primary_keys_filtered(primary_keys_path)
    primary_taglist = [v for _, v in osm_schema.TagKey.items()]
    
    proj = setup_tiler(utm_zone)
    parent_tile = proj.tile_from_quadkey(quadkey)

    osm_file = COG_PATH + f'''/{quadkey}/{osm_filename}'''
    
    egm, lut, tree = build_osm_rtree(osm_file)
    tile_children = get_children_at_zoom([parent_tile], dst_zoom)
    
    record_file = COG_PATH + f'''/{quadkey}/qk_label_record.csv'''
    with open(record_file, "w") as rf:
        writer = csv.writer(rf)
        writer.writerow(["TileQuadkey", "PrimaryKey", "KeyValue", "OSMID", "ElmType"])
        n=0
        for ctile in tile_children:
            ctile.toWGS84()
            g = geom.shape(ctile)
            results = [(lut[id(elm_geom)], elm_geom.wkt) for elm_geom in tree.query(g)]
            for osm_id, wktgeom in results:
                osmd = egm[osm_id][-1]
                tags = osmd['tags']
                ptags, vals = zip(*[(k, v) for k, v in tags.items() if k in primary_taglist])
                if len(ptags) > 1:
                    ptags = "; ".join(ptags)
                    vals = "; ".join(vals)
                elif len(ptags) == 0:
                    continue
                else:
                    ptags = ptags[0]
                    vals = vals[0]

                writer.writerow([ctile.quadkey, ptags, vals, osm_id, osmd['type']])
                n += 1
    print(f'''wrote {n} rows for quadkey {quadkey}''')
    return record_file
        
                 

In [9]:
#dd = vx.open('/home/ubuntu/data/ard/33/*/qk_label_record.csv', dtype=str, convert="/home/ubuntu/data/master_label_record.hdf5")
dd_filt = vx.open('/home/ubuntu/data/master_label_record_ndfiltered.hdf5')

In [29]:
dd.sample(n=100, random_state=42)

#,TileQuadkey,PrimaryKey,KeyValue,OSMID,ElmType
0,01331330320202201,'addr:housenumber; addr:postcode; addr:street; a...,1; 16775; Friedrichsthaler Weg; Nassenheide,3850738657,node
1,01331332121330103,building,shed,317071074,way
2,01331332103230213,'addr:housenumber; addr:postcode; addr:street; a...,36K; 10405; Prenzlauer Allee; Prenzlauer Berg,3042132619,node
3,01331332122322031,waterway,ditch,246475785,way
4,01331332031130131,bicycle; foot; highway; name,yes; yes; track; Schildhornweg,31435564,way
...,...,...,...,...,...
95,01331332120201221,highway,path,107235990,way
96,01331332102110330,amenity,parking,32386625,way
97,01331332103222233,highway,footway,254523905,way
98,01331332103012031,building,yes,418321938,way


In [ ]:
# Almost 3 million ground truths!

In [58]:
#dd_filt.export_hdf5("/home/ubuntu/data/master_label_record_ndfiltered.hdf5")

In [27]:
del dd_filt

In [28]:
building_quads = gds.mor[gds.mor.PrimaryKey.str.contains("building") & gds.mor.KeyValue.str.contains("yes")].TileQuadkey.unique()
len(building_quads)

30519

30519

In [35]:
class GlobalDataSecretary:
    def __init__(self,
                 data_path="/home/ubuntu/data",
                 label_schema="labels.json",
                 nodata_index="chips/33/nodata_index.json",
                 master_osm_record="master_label_record_ndfiltered.hdf5",
                 ):
        
        self.data_path = data_path
        if label_schema:
            with open(os.path.join(self.data_path, label_schema)) as s:
                self.label_schema = json.load(s)
        
        if nodata_index:
            p = os.path.join(self.data_path, nodata_index)
            print(p)
            with open(p) as s:
                self.features = json.load(s)["all_data"]
        
        self.mor = vx.open(os.path.join(self.data_path, master_osm_record))
        self._qkf_map = None
        self._qkl_map = None
        self.dense_enc = None
        self.sparse_enc = None
        
    @property
    def qkfeature_map(self):
        "quadkeys -> [feature1, ...]"
        if not self._qkf_map:
            self._qkf_map = collections.defaultdict(list)
            for ftr in self.features:
                self._qkf_map[ftr[4:21]].append(ftr)
        return self._qkf_map
    
    @property
    def qklabel_map(self):
        "quadkeys -> [label1, ...]"
        if not self._qkl_map:
            self._qkl_map = collections.defaultdict(list)
        return self._qkl_map
        
        
    def filter_record_by_label(self, label=None, schema=None):
        if not schema:
            schema = self.label_schema[label]
        containers = []
        for key, vals in schema.items():
            vals_regex = "|".join(vals)
            qk_containers = self.mor[self.mor.PrimaryKey.str.contains(key) & self.mor.KeyValue.str.contains(vals_regex)].TileQuadkey.unique()
            containers.extend(qk_containers)
        containers = list(set(containers))
        for qk in containers:
            self.qklabel_map[qk].append(label)
        
    
    def filter_record_by_qk(self, retain=None):
        self.mor_f = self.mor[self.mor.TileQuadkey.isin(retain)]
        return self.mor_f
    
    def dense_encode_labels(self, label_schema=None, overwrite=False, update=True):
        pass
        
        
        
    
        
            
        

In [36]:
gds = GlobalDataSecretary()

/home/ubuntu/data/chips/33/nodata_index.json


In [29]:
grass = gds.label_schema['Grassland']

In [31]:
grass_quads = gds.mor[gds.mor.PrimaryKey.str.contains("natural") & gds.mor.KeyValue.str.contains("heath|grassland|scrub")]
grass_quads

#,TileQuadkey,PrimaryKey,KeyValue,OSMID,ElmType
0,01331330231232131,natural,scrub,381437605,way
1,01331330231232133,natural,scrub,381186374,way
2,01331330231232133,natural,scrub,381437605,way
3,01331330231232300,landuse; leaf_type; natural,forest; mixed; grassland,493042596,way
4,01331330231232301,landuse; leaf_type; natural,forest; mixed; grassland,493042596,way
...,...,...,...,...,...
"24,417",01331332301103122,natural,scrub,111869022,way
"24,418",01331332301122220,natural,scrub,839852966,way
"24,419",01331332301122220,natural,scrub,839852965,way
"24,420",01331332301122222,natural,scrub,839852966,way


In [38]:
gds.filter_record_by_label("Grassland")

In [15]:
len(gds.qklabel_map)

22719

In [16]:
gds.label_schema.keys()

dict_keys(['Grassland', 'Wood', 'Forest', 'Residential', 'Railway', 'Building', 'House', 'Road', 'Water', 'Parking', 'Pool', 'Park', 'Soccer'])

In [40]:
gds.filter_record_by_label("Wood")
print(len(gds.qklabel_map))

32004


In [41]:
gds.filter_record_by_label("Forest")
print(len(gds.qklabel_map))

42222


In [42]:
gds.filter_record_by_label("Residential")
print(len(gds.qklabel_map))

57060


In [43]:
gds.filter_record_by_label("Railway")
print(len(gds.qklabel_map))

61353


In [44]:
gds.filter_record_by_label("Building")
print(len(gds.qklabel_map))

62162


In [45]:
gds.filter_record_by_label("House")
print(len(gds.qklabel_map))

62186


In [46]:
gds.filter_record_by_label("Road")
print(len(gds.qklabel_map))

69898


In [47]:
gds.filter_record_by_label("Water")
print(len(gds.qklabel_map))

70492


In [48]:
gds.filter_record_by_label("Parking")
print(len(gds.qklabel_map))

70500


In [49]:
gds.filter_record_by_label("Pool")
print(len(gds.qklabel_map))

70500


In [50]:
gds.filter_record_by_label("Park")
print(len(gds.qklabel_map))

70538


In [51]:
gds.filter_record_by_label("Soccer")
print(len(gds.qklabel_map))

70544


In [57]:
with open("/home/ubuntu/data/dense_encodings.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["Feature", "Labels"])
    for qk, labels in gds.qklabel_map.items():
        labels = list(set(labels))
        dense_labels = " ".join(labels)
        for ftr in gds.qkfeature_map[qk]:
            writer.writerow([ftr, dense_labels])
        

In [58]:
dense = vx.from_csv("/home/ubuntu/data/dense_encodings.csv")
dense

#,Feature,Labels
0,Z33-01331330233122333_1040010052D58500.jpg,Road Wood Forest Railway Water Grassland
1,'Z33-01331330233122333_06480218-b97f-4761-ab0f-4...,Road Wood Forest Railway Water Grassland
2,'Z33-01331332032130120_06480218-b97f-4761-ab0f-4...,'Park Wood Road Building Pool Parking Residentia...
3,'Z33-01331332012301121_06480218-b97f-4761-ab0f-4...,Building Residential Wood House
4,'Z33-01331332102313023_01280972-72af-433b-9b2c-c...,Wood Road Building Parking Residential Soccer
...,...,...
"112,780",Z33-01331332013302013_1040010052D58500.jpg,Soccer
"112,781",'Z33-01331332013302013_06480218-b97f-4761-ab0f-4...,Soccer
"112,782",Z33-01331332100131031_10400100549CFC00.jpg,Soccer
"112,783",'Z33-01331332100131031_b8b3ff19-2d8b-41d6-bd48-2...,Soccer


In [56]:
del dense


In [ ]:
gds.qkfeature_map

In [61]:
def build_chip_map(img_chips=None):
    if not img_chips:
        img_chips = load_nodata_scores()['all_data']
    qkimg_map = collections.defaultdict(list)
    for img_chip in img_chips:
        qkimg_map[img_chip[4:21]].append(img_chip)
    return qkimg_map    

In [62]:
qkimg_map = build_chip_map()


71666

In [50]:
def filter_nodata_from_record():
    valid_chips = load_nodata_scores()['all_data']
    valid_qks = list(set([f[4:21] for f in valid_chips]))
    qks_to_filter = set(dd.TileQuadkey.unique()).intersection(set(valid_qks))
    return list(qks_to_filter)

In [51]:
qks_to_filter = filter_nodata_from_record()
len(qks_to_filter)

71664

In [52]:
dd_filt = dd[dd.TileQuadkey.isin(qks_to_filter)]

In [54]:
len(dd_filt)

2595186

In [56]:
len(dd)

2821690

2821690

32981

In [26]:
labels.keys()

dict_keys(['Grassland', 'Wood', 'Forest', 'Residential', 'Railway', 'Building', 'House', 'Road', 'Water', 'Parking', 'Pool', 'Park', 'Soccer'])

In [12]:
pkeys = itertools.chain.from_iterable([vals.keys() for lbl, vals in labels.items()])

In [13]:
pkeys = list(set(pkeys))
pkeys

['landuse',
 'building',
 'railway',
 'amenity',
 'waterway',
 'leisure',
 'natural',
 'highway']

In [27]:
labels

{'Grassland': {'natural': ['heath', 'grassland', 'scrub'],
  'landuse': ['meadow']},
 'Wood': {'natural': ['wood', 'tree', 'deciduous']},
 'Forest': {'landuse': ['forest']},
 'Residential': {'landuse': ['residential'],
  'building': ['residential'],
  'highway': ['residential']},
 'Railway': {'landuse': ['railway'], 'railway': ['rail']},
 'Building': {'building': ['yes']},
 'House': {'building': ['house']},
 'Road': {'highway': ['motorway',
   'trunk',
   'primary',
   'secondary',
   'tertiary',
   'track',
   'unclassified']},
 'Water': {'waterway': ['river'], 'natural': ['water']},
 'Parking': {'amenity': ['parking']},
 'Pool': {'leisure': ['pool']},
 'Park': {'leisure': ['park']},
 'Soccer': {'leisure': ['pitch']}}

In [ ]:
df = vaex.open("/home/ubuntu/data/ard/33/*/training_data.csv", convert="/home/ubuntu/data/training_data.hdf5")

In [ ]:
labels = dict()
plt.figure(figsize=(16, 4))
r.plot(kind='bar')

In [ ]:
natural = qklbls[qklbls.PrimaryKey.str.contains("natural")]
natural

In [ ]:
plt.figure(figsize=(12, 4))
natural.KeyValue.value_counts().plot(kind='bar', title="natural")

In [10]:
labels["Grassland"] = {"natural": ["heath", "grassland", "scrub"]}
labels["Wood"] = {"natural": ["wood", "tree", "deciduous"]}

In [ ]:
landuse = qklbls[qklbls.PrimaryKey.str.contains("landuse")]
landuse

In [ ]:
plt.figure(figsize=(12, 4))
landuse.KeyValue.value_counts().plot(kind='bar', title="landuse")

In [11]:
labels["Grassland"]["landuse"] = ["meadow"]
labels["Forest"] = {"landuse": ["forest"]}
labels["Residential"] = {"landuse": ["residential"]}
labels["Railway"] = {"landuse": ["railway"]}
labels

{'Grassland': {'natural': ['heath', 'grassland', 'scrub'],
  'landuse': ['meadow']},
 'Wood': {'natural': ['wood', 'tree', 'deciduous']},
 'Forest': {'landuse': ['forest']},
 'Residential': {'landuse': ['residential']},
 'Railway': {'landuse': ['railway']}}

In [ ]:
building = qklbls[qklbls.PrimaryKey.str.contains("building")]
building

In [ ]:
plt.figure(figsize=(12, 4))
building.KeyValue.value_counts().head(20).plot(kind='bar', title="building")

In [12]:
labels["Building"] = {"building": ["yes"]}
labels["Residential"]["building"] = ["residential"]
labels["House"] = {"building": ["house"]}
labels

{'Grassland': {'natural': ['heath', 'grassland', 'scrub'],
  'landuse': ['meadow']},
 'Wood': {'natural': ['wood', 'tree', 'deciduous']},
 'Forest': {'landuse': ['forest']},
 'Residential': {'landuse': ['residential'], 'building': ['residential']},
 'Railway': {'landuse': ['railway']},
 'Building': {'building': ['yes']},
 'House': {'building': ['house']}}

In [ ]:
highway = qklbls[qklbls.PrimaryKey.str.contains("highway")]
highway

In [ ]:
plt.figure(figsize=(12, 4))
highway.KeyValue.value_counts().head(20).plot(kind='bar', title="highway")

In [13]:
labels["Residential"]["highway"] = ["residential"]
labels["Road"] = {"highway": ["motorway", "trunk", "primary", "secondary", "tertiary", "track", "unclassified"]}
labels

{'Grassland': {'natural': ['heath', 'grassland', 'scrub'],
  'landuse': ['meadow']},
 'Wood': {'natural': ['wood', 'tree', 'deciduous']},
 'Forest': {'landuse': ['forest']},
 'Residential': {'landuse': ['residential'],
  'building': ['residential'],
  'highway': ['residential']},
 'Railway': {'landuse': ['railway']},
 'Building': {'building': ['yes']},
 'House': {'building': ['house']},
 'Road': {'highway': ['motorway',
   'trunk',
   'primary',
   'secondary',
   'tertiary',
   'track',
   'unclassified']}}

In [ ]:
waterway = qklbls[qklbls.PrimaryKey.str.contains("waterway")]
waterway

In [ ]:
plt.figure(figsize=(12, 4))
waterway.KeyValue.value_counts().head(30).plot(kind='bar', title="waterway")

In [14]:
labels["Water"] = {"waterway": ["river"], "natural": ["water"]}
labels

{'Grassland': {'natural': ['heath', 'grassland', 'scrub'],
  'landuse': ['meadow']},
 'Wood': {'natural': ['wood', 'tree', 'deciduous']},
 'Forest': {'landuse': ['forest']},
 'Residential': {'landuse': ['residential'],
  'building': ['residential'],
  'highway': ['residential']},
 'Railway': {'landuse': ['railway']},
 'Building': {'building': ['yes']},
 'House': {'building': ['house']},
 'Road': {'highway': ['motorway',
   'trunk',
   'primary',
   'secondary',
   'tertiary',
   'track',
   'unclassified']},
 'Water': {'waterway': ['river'], 'natural': ['water']}}

In [ ]:
railway = qklbls[qklbls.PrimaryKey.str.contains("railway")]
railway

In [ ]:
plt.figure(figsize=(12, 4))
railway.KeyValue.value_counts().head(30).plot(kind='bar', title="railway")

In [15]:
labels["Railway"]["railway"] = ["rail"]
labels

{'Grassland': {'natural': ['heath', 'grassland', 'scrub'],
  'landuse': ['meadow']},
 'Wood': {'natural': ['wood', 'tree', 'deciduous']},
 'Forest': {'landuse': ['forest']},
 'Residential': {'landuse': ['residential'],
  'building': ['residential'],
  'highway': ['residential']},
 'Railway': {'landuse': ['railway'], 'railway': ['rail']},
 'Building': {'building': ['yes']},
 'House': {'building': ['house']},
 'Road': {'highway': ['motorway',
   'trunk',
   'primary',
   'secondary',
   'tertiary',
   'track',
   'unclassified']},
 'Water': {'waterway': ['river'], 'natural': ['water']}}

In [ ]:
amenity = qklbls[qklbls.PrimaryKey.str.contains("amenity")]
amenity

In [ ]:
plt.figure(figsize=(12, 4))
amenity.KeyValue.value_counts().head(30).plot(kind='bar', title="amenity")

In [16]:
labels["Parking"] = {"amenity": ["parking"]}
labels

{'Grassland': {'natural': ['heath', 'grassland', 'scrub'],
  'landuse': ['meadow']},
 'Wood': {'natural': ['wood', 'tree', 'deciduous']},
 'Forest': {'landuse': ['forest']},
 'Residential': {'landuse': ['residential'],
  'building': ['residential'],
  'highway': ['residential']},
 'Railway': {'landuse': ['railway'], 'railway': ['rail']},
 'Building': {'building': ['yes']},
 'House': {'building': ['house']},
 'Road': {'highway': ['motorway',
   'trunk',
   'primary',
   'secondary',
   'tertiary',
   'track',
   'unclassified']},
 'Water': {'waterway': ['river'], 'natural': ['water']},
 'Parking': {'amenity': ['parking']}}

In [ ]:
leisure = qklbls[qklbls.PrimaryKey.str.contains("leisure")]
leisure

In [ ]:
plt.figure(figsize=(12, 4))
leisure.KeyValue.value_counts().plot(kind='bar', title="leisue")

In [17]:
labels["Pool"] = {"leisure": ["pool"]}
labels["Park"] = {"leisure": ["park"]}
labels["Soccer"] = {"leisure": ["pitch"]}
labels

{'Grassland': {'natural': ['heath', 'grassland', 'scrub'],
  'landuse': ['meadow']},
 'Wood': {'natural': ['wood', 'tree', 'deciduous']},
 'Forest': {'landuse': ['forest']},
 'Residential': {'landuse': ['residential'],
  'building': ['residential'],
  'highway': ['residential']},
 'Railway': {'landuse': ['railway'], 'railway': ['rail']},
 'Building': {'building': ['yes']},
 'House': {'building': ['house']},
 'Road': {'highway': ['motorway',
   'trunk',
   'primary',
   'secondary',
   'tertiary',
   'track',
   'unclassified']},
 'Water': {'waterway': ['river'], 'natural': ['water']},
 'Parking': {'amenity': ['parking']},
 'Pool': {'leisure': ['pool']},
 'Park': {'leisure': ['park']},
 'Soccer': {'leisure': ['pitch']}}

In [31]:
with open("/home/ubuntu/data/labels.json", "w") as f:
    json.dump(labels, f)

01331330232300332


In [ ]:
ndf = pd.DataFcoordsrom_records(osm_nodes)
ndf = gpd.GeoDataFrame(ndf, geometry=gpd.points_from_xy(ndf.lon, ndf.lat), crs=WORLD_CRS)
#ndf.set_crs(crs=WORLD_CRS, inplace=True)
ndf.head()

In [ ]:
parent_tile = proj.tile_from_quadkey(qk)
chillun_tiles = get_children_at_zoom([parent_tile], 17)
chide = chillun_tiles[0]
ndf.to_crs(crs=chide.crs, inplace=True)
ndf.head()

In [ ]:
qkdf = {"quadkey": [c.quadkey for c in chillun_tiles], "geometry": [geom.shape(c) for c in chillun_tiles]}
qkdf = gpd.GeoDataFrame(qkdf, crs=chide.crs)
qkdf.head()

In [ ]:
%%time
nqkdf = gpd.sjoin(ndf, qkdf, how="inner", op="intersects")
nqkdf

In [ ]:
def close_viswins():
    vis.close(imgrid)
    vis.close(properties_window)

def load_chip_array(data_score="all_data", n=12, indices=None, quality_map=ndix, chip_path=CHIP_PATH):
    if not indices:
        sample_space = range(0, len(ndix[data_score]) - 1)
        indices = random.sample(sample_space, n)
    images = [io.imread(os.path.join(chip_path, quality_map[data_score][idx])) for idx in indices]
    return [img.transpose(2, 0, 1) for img in images]

images = load_chip_array()


imgrid_opts = dict(title='chip_sample_viewer', caption="whatever")
imgrid = vis.images(images, padding=4, nrow=4, opts=imgrid_opts)

#vis.close(imgrid)


properties = [
    {'type': 'number', 'name': 'Image sample size', 'value': '12'},
    {'type': 'button', 'name': 'Button', 'value': 'Resample'},
    {'type': 'select', 'name': 'Select', 'value': "1", 'values': ["all_data", "partial_data", "no_data"]},
]

properties_window = vis.properties(properties, env='main')

def properties_callback(event):
    if event['event_type'] == 'PropertyUpdate':
        prop_id = event['propertyId']
        value = event['value']
        new_value = value
        if prop_id != 1:
            properties[prop_id]['value'] = new_value
        vis.properties(properties, win=properties_window)
        score_idx = int(properties[2]['value'])
        data_score = properties[2]['values'][score_idx]
        images = load_chip_array(data_score=data_score, n=int(properties[0]['value']))
        vis.images(images, padding=4, nrow=4, opts=imgrid_opts, win=imgrid)
        

vis.register_event_handler(properties_callback, properties_window)

In [38]:
class SkywayDataCatalog:
    _tiler = WNUTM5kmTiling()
    _qkp_root = qkp
    
    def __init__(self, 
                 utm_zone=33,
                 tilezoom=17, 
                 sources_basepath=DATA_PATH,
                 outputs_basepath="/home/ubuntu/data/chips",
                 input_nominatim="primary_osm_data_filtered.json",
                 nodata_index="nodata_index.json",
                 quadkeys=None,
                 ):
        
        self.utm_zone = utm_zone
        self.tilezoom = tilezoom
        self.sources_path = os.path.join(sources_basepath, str(utm_zone))
        self.outputs_path = os.path.join(outputs_basepath, str(utm_zone))
        self._quadkeys = quadkeys
        
        with open(os.path.join(self.outputs_path, nodata_index)) as f:
            self.image_tiles = json.load(f).get("all_data")
        
        self._qkp_child = quadkey_regex(zoom=tilezoom)
        self.tiler = ProjectedUTMTiling(utm_zone, self._tiler)
        self.__gi__ = None
    
    @property
    def qkroots(self):
        if not self._quadkeys:
            with os.scandir(self.sources_path) as it:
                self._quadkeys = [qk.name for qk in it if self._qkp_root.match(qk.name)]
        return self._quadkeys
    
    @property
    def num_qks(self):
        return len(self.qkroots)
    
    @property
    def area_coverage(self, units='m'):
        return str((5_000 * 5_000) * self.nqks) + ' ' + units + '^2'
    
    def __getitem__(self, quadkey):
        if quadkey not in self.qkroots:
            raise KeyError("Out of bounds")
        return self.tiler.tile_from_quadkey(quadkey)
    
    def __iter__(self):
        tiles = np.array([self.tiler._tiler.quadkey_to_tile(qk)
                      for qk in self.qkroots], dtype=GeoTile.dtype)
        for tile in np.sort(tiles, order=['iy', 'ix']):
            yield self.tiler._tf(tt.base.Tile(*tile))
    
    def iter_geoms(self):
        for tile in iter(self):
            with WGS84Transformer(tile) as geotile:
                yield geotile.asShape
            
    @property
    def __geo_interface__(self):
        if self.__gi__ is None:
            self.__gi__ = itrreduce(ops.unary_union,
                                    self.iter_geoms()).__geo_interface__
        return self.__gi__
    
    @property
    def asShape(self):
        return geom.asShape(self)
    
    @property
    def centroid(self):
        return self.asShape.centroid
    
    @property
    def map_center(self):
        return (self.centroid.y, self.centroid.x)
    
    def get_geojson(self):
        fc = {"type": "FeatureCollection",
              "features": list()
             }
    
        for qkroot_tile in iter(self):
            with WGS84Transformer(qkroot_tile) as t:
                fc["features"].append(to_gjson(t))
        
        return fc
    

    
            
    
sdc = SkywayDataCatalog()

In [ ]:
from rasterio.plot import show as rashow


def overview(impath):
    with rasterio.open(impath) as src:
        plt.figure(figsize=(10,10))
        arr = src.read(out_shape=(3, int(src.height / 16), int(src.width / 16)))[..., 32:1056, 32:1056]
        rashow(arr)
    return arr
    
class TileViewer:
    def __init__(self, tc, qk):
        self.tc = tc
        self.qk = qk
        self.path = tc.qk_path + "/{}".format(qk)
        self.tifs = [f for f in os.listdir(self.path) if f[-4:] == ".tif"]
        self.fpaths = [self.path + "/" + tif for tif in self.tifs]

In [ ]:

try:
    del m
except:
    pass

# %%time


m = Map(center=sdc.map_center, zoom=8)
#m.clear_layers()
m.add_layer(TileLayer(url=tms_url))

geo_json = GeoJSON(data=sdc.get_geojson(), style = {'color': 'red', 'opacity':1, 'weight':1.9, 'fillOpacity':0.1})

html = widgets.HTML('''Hover over a tile''')
html.layout.margin = '0px 20px 20px 20px'
control = WidgetControl(widget=html, position='topright')
m.add_control(control)

def update_html(feature, **kwargs):
    html.value = '''
    <h3><b>{}</b><h3>
    <h4>Tile Index: ({}, {})</h4>
    '''.format(feature['properties']['tile_id'],
               feature['properties']['ix'],
               feature['properties']['iy'])
    
geo_json.on_hover(update_html)

sc = Sidecar(title="whatever")
with sc:
    display(m)

In [ ]:
from ipyleaflet import TileLayer

In [ ]:
tms_url = 'http://localhost:8000/{z}/{x}/{y}'
bm = {'url': tms_url, 'attribution': 'A big Butt'}

In [ ]:
m.add_layer(geo_json);


In [ ]:
ardtiles = basemap_to_tiles(bm)

In [ ]:
m.add_layer(ardtiles)

In [ ]:
m.clear_layers()
del m

In [ ]:
display(basemaps)

In [ ]:
basemap_to_tiles??

In [ ]:
from rio_tiler.io.cogeo import COGReader
from cogeo_mosaic.mosaic import MosaicJSON
from cogeo_mosaic.backends import MosaicBackend
import mercantile

from rio_tiler_mosaic.mosaic import mosaic_tiler
from rio_tiler_mosaic.methods import defaults

from rio_tiler_mosaic.methods.base import MosaicMethodBase

from rio_tiler.profiles import img_profiles


from rasterio.plot import reshape_as_image

#mosaic_def = MosaicJSON.from_urls(cog_paths)
#with MosaicBackend("/home/ubuntu/data/ard/33/mosaic.json", mosaic_def=mosaic_def) as mos:
#    mos.write(overwrite=True)
    
mosaic = MosaicBackend("/home/ubuntu/data/ard/33/mosaic.json")

mtile = mercantile.quadkey_to_tile('120210232133')

(arr, mask), assets = mosaic.tile(mtile.x, mtile.y, mtile.z)

fig = plt.figure(figsize=(30, 10))

ax = fig.add_subplot(1, 2, 1)
ax.imshow(reshape_as_image(arr))

ax = fig.add_subplot(1, 2, 2)
ax.imshow(mask)

In [ ]:
mtile

In [ ]:
mosaic.reader_options